In [4]:
import spotipy
import spotipy.util as util
import numpy as np
import pandas as pd

In [5]:
absolute_path = "f:\\AI Projects\\Music Classification"
relative_path = "Data\\Spotify"
full_path = os.path.join(absolute_path, relative_path)

In [6]:
CLIENT_ID = "372d9bfe8dce48a1aac1806d4329629d"
CLIENT_SECRET = "1a6e5a340428469dba631f9ffbd4528f"

In [7]:
auth_manager = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [8]:
def get_playlist_tracks(playlist_id):
    playlist_data = sp.playlist_tracks(playlist_id)
    tracks = playlist_data['items']
    while playlist_data['next']:
        playlist_data = sp.next(playlist_data)
        tracks.extend(playlist_data['items'])
        
    return tracks

Get tracks from playlists

In [9]:
playlists = ["2MepDCzimtCz2ytAlLUdUi","1h0CEZCm6IbFTbxThn6Xcs","37i9dQZF1DX8qqIDAkKiQg","7pWNG89bI9uwHQaUI3pINb","37i9dQZF1DX12YIlzNkccL","2hRc6PoqD046cyS8QTFKkQ"] #["4rnleEAOdmFAbRcNCgZMpY","37i9dQZF1DWWEJlAGA9gs0","37i9dQZF1DXbITWG1ZJKYt","37i9dQZF1DX1spT6G94GFC","5oM9tyhspzK18TrPrlIZro","37i9dQZF1DX55yuR78Invt"]

In [11]:
import warnings
warnings.filterwarnings('ignore')

columns = ["artist","album","track_name","track_id","released_date","added_date","genres","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms","time_signature"]

metadata_playlists = pd.DataFrame(columns=columns)

for playlist in playlists:
    playlist_data = get_playlist_tracks(playlist)
    for track in playlist_data:
        artist = track["track"]["album"]["artists"][0]["name"]
        artist_uri = track["track"]["album"]["artists"][0]["uri"]
        album = track["track"]["album"]["name"]
        track_name = track["track"]["name"]
        if not (
            metadata_playlists['artist'].str.contains(artist, regex=False).any() & 
            metadata_playlists['album'].str.contains(album, regex=False).any() & 
            metadata_playlists['track_name'].str.contains(track_name, regex=False).any()):
            track_id = track["track"]["id"]
            released_date = track["track"]["album"]["release_date"]
            added_date = track["added_at"]
            genres = sp.artist(artist_uri)["genres"]
            new_row = {"artist" : artist, "album" : album, "track_name" : track_name, "track_id" : track_id,"released_date" : released_date, "added_date" : added_date,"genres" : genres}
            audio_features = sp.audio_features(track_id)[0]
            for feature in columns[7:]:
                new_row[feature] = audio_features[feature]
            metadata_playlists = metadata_playlists.append(new_row, ignore_index=True)
        else:
            print(f"Skipping: {track_name} due to a duplicate")


Every Breath You Take
Sweet Child O' Mine
Too Many Pieces
Your Love
Africa
Gucci
Livin' On A Prayer
The Final Countdown
Eye of the Tiger
Nothing's Gonna Stop Us Now
Back In Black
I Want to Know What Love Is - 1999 Remaster
Don't Stop Believin'
Patience
Another Day in Paradise - 2016 Remaster
Paradise City
Heaven
She's Like the Wind (feat. Wendy Fraser)
Welcome To The Jungle
Walk Of Life
I'm Still Standing
Don't You (Forget About Me) - Remastered 2001
I Still Haven't Found What I'm Looking For
Alone
Listen To Your Heart
You Shook Me All Night Long
Maneater
Burning Heart - From "Rocky IV" Soundtrack
Don't Dream It's Over
Never Tear Us Apart
Jump - 2015 Remaster
You Give Love A Bad Name
Carry on Wayward Son
Everytime You Go Away
Bette Davis Eyes
I Love Rock 'n Roll
Faithfully
Purple Rain
With Or Without You - Remastered
Shout
Tougher Than the Rest
Beds Are Burning - Remastered
Poison
How Am I Supposed to Live Without You
I Hate Myself for Loving You
Who Made Who
Can't Fight This Feeling
I

In [ ]:
metadata_playlists.head()

In [ ]:
metadata_playlists.to_csv(os.path.join(full_path,"metadata_playlists_1.csv"))

Get top tracks from artists

In [9]:
artists = ["spotify:artist:36QJpDe2go2KgaRleHCDTp",
"spotify:artist:6mdiAmATAx73kdxrNrnlao",
"spotify:artist:2ye2Wgw4gimLv2eAKyk1NB",
"spotify:artist:58lV9VcRSjABbAbfWS6skp",
"spotify:artist:27T030eWyCQRmDyuvr1kxY",
"spotify:artist:26bcq2nyj5GB7uRr558iQg",
"spotify:artist:6olE6TJLqED3rqDCT0FyPh",
"spotify:artist:0rvjqX7ttXeg3mTy8Xscbt",
"spotify:artist:711MCceyCBcFnzjGY4Q7Un",
"spotify:artist:3qm84nBOXUEQ2vnTfUTTFC",
"spotify:artist:7Js6Lde8thlIHXggv2SCBz"]

In [11]:
import warnings
warnings.filterwarnings('ignore')

columns = ["artist","album_name","track_name","track_id","released_date","explict","genres","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms","time_signature"]

metadata_albums = pd.DataFrame(columns=columns)

for artist_uri in artists:
    artist = sp.artist(artist_uri)["name"]
    results = sp.artist_albums(artist_uri, album_type='album')
    albums = results['items']
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    for album in albums:
        album_name = album["name"]
        released_date = album["release_date"]
        tracks = sp.album_tracks(album["id"])      
        for track in tracks["items"]:
            track_name = track["name"]
            if not (
                metadata_albums['artist'].str.contains(artist, regex=False).any() & 
                metadata_albums['album_name'].str.contains(album_name, regex=False).any() & 
                metadata_albums['track_name'].str.contains(track_name, regex=False).any()):
                track_id = track["id"]
                explict = track["explicit"]
                genres = sp.artist(artist_uri)["genres"]
                new_row = {"artist" : artist, "album_name" : album_name, "track_name" : track_name, "track_id" : track_id,"released_date" : released_date,"explict" : explict,"genres" : genres}
                audio_features = sp.audio_features(track_id)[0]
                for feature in columns[7:]:
                    new_row[feature] = audio_features[feature]
                metadata_albums = metadata_albums.append(new_row, ignore_index=True)
            else:
                print(f"Skipping: {track_name} due to a duplicate")

Skipping: Trampled Under Foot - Remaster due to a duplicate
Skipping: Houses of the Holy - Remaster due to a duplicate
Skipping: Kashmir - Remaster due to a duplicate
Skipping: Poor Tom - Remaster due to a duplicate
Skipping: I Can't Quit You Baby - Remaster due to a duplicate
Skipping: Walter's Walk - Remaster due to a duplicate
Skipping: Ozone Baby - Remaster due to a duplicate
Skipping: Darlene - Remaster due to a duplicate
Skipping: Bonzo's Montreux - Remaster due to a duplicate
Skipping: Wearing and Tearing - Remaster due to a duplicate
Skipping: All My Love - Remaster due to a duplicate
Skipping: South Bound Saurez - Remaster due to a duplicate
Skipping: Fool in the Rain - Remaster due to a duplicate
Skipping: Hot Dog - Remaster due to a duplicate
Skipping: Carouselambra - Remaster due to a duplicate
Skipping: All My Love - Remaster due to a duplicate
Skipping: I'm Gonna Crawl - Remaster due to a duplicate
Skipping: Nobody's Fault but Mine - Remaster due to a duplicate
Skipping: 

TypeError: 'NoneType' object is not subscriptable

In [15]:
metadata_albums.head()

,artist,album_name,track_name,track_id,released_date,explict,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Led Zeppelin,The Complete BBC Sessions (Remastered),You Shook Me - 23/3/69 Top Gear; Remaster,4AIJz1t4ysqOT1c5BLSRQQ,2016-09-16,False,"[album rock, classic rock, hard rock, rock]",0.414,0.490,4,-8.576,1,0.0545,0.12700,0.09990,0.472,0.670,130.761,314013,3
1,Led Zeppelin,The Complete BBC Sessions (Remastered),I Can't Quit You Baby - 23/3/69 Top Gear;Remaster,5QRD5sNh0aaWMyjTzQ0QIn,2016-09-16,False,"[album rock, classic rock, hard rock, rock]",0.411,0.442,2,-12.276,1,0.2620,0.02300,0.02660,0.198,0.367,145.973,263627,3
2,Led Zeppelin,The Complete BBC Sessions (Remastered),Communication Breakdown - Live on Tasty Pop Su...,0RR8wuHHc5NqSFxhPDDBNV,2016-09-16,False,"[album rock, classic rock, hard rock, rock]",0.367,0.779,9,-8.334,1,0.0488,0.00574,0.00708,0.288,0.794,90.354,191827,4
3,Led Zeppelin,The Complete BBC Sessions (Remastered),Dazed and Confused - 3/23/69 Top Gear;Remaster,3vDA1z8UmHtVLQV7McvhEj,2016-09-16,False,"[album rock, classic rock, hard rock, rock]",0.303,0.571,11,-11.521,0,0.0958,0.00882,0.07570,0.114,0.422,152.001,399587,3
4,Led Zeppelin,The Complete BBC Sessions (Remastered),The Girl I Love She Got Long Black Wavy Hair -...,7ecVrUYlhj6OrKTAK0oDzo,2016-09-16,False,"[album rock, classic rock, hard rock, rock]",0.264,0.609,9,-10.992,0,0.0655,0.11100,0.31100,0.322,0.862,185.875,183227,4


In [14]:
metadata_albums = metadata_albums[~metadata_albums['artist'].isin(['Journey'])]

In [17]:
metadata_albums.to_csv(os.path.join(full_path,"metadata_albums_2.csv"))